In [137]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import random
import numpy as np

In [ ]:
def set_seed(seed=42):

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) 
    np.random.seed(seed)
    random.seed(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)


In [139]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
print("Using device", device)

Using device cuda


In [140]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,),(0.5))])

train_data_ = datasets.FashionMNIST(root='./data', train = True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root='./data', train = False, download=True, transform=transform)

train_len = int(0.9*len(train_data_))
val_len = len(train_data_)-train_len
train_data, val_data = random_split(train_data_, (train_len, val_len))

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [ ]:
class Fashion(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),

            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),  
            nn.Linear(64, 10),
        )   

    def forward(self, x):
            return self.model(x)

In [ ]:
Fashion_ANN = Fashion().to(device)
Loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(Fashion_ANN.parameters(), lr=1e-3)   
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,      
    mode='min',                      
    patience=4,    
    factor=0.5      
)

In [ ]:
epochs = 30
es_patience = 10
counter = 0
best_val_loss = float('inf')
delta = 1e-4

for epoch in range(epochs):
    Fashion_ANN.train()
    total_loss=0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = Fashion_ANN(images)
        loss = Loss(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() 
    avg_loss = total_loss/len(train_loader)

    #Validation
    Fashion_ANN.eval()
    total_val_loss = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = Fashion_ANN(images)
            val_loss = Loss(outputs, labels).item()
            total_val_loss += val_loss
    
    avg_val_loss = total_val_loss/len(val_loader)
    scheduler.step(avg_val_loss)   
    #Early Stopping Check
    if avg_val_loss < best_val_loss-delta:
        best_val_loss = avg_val_loss
        counter = 0
        torch.save(Fashion_ANN.state_dict(), "best_model.pth")
    else:
        counter+=1
        if counter>=es_patience:
            print("Early stopping triggered.")
            break
        
    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Counter: {counter}")

Epoch 1/30 | Train Loss: 0.5280 | Val Loss: 0.3833 | Counter: 0
Epoch 2/30 | Train Loss: 0.3811 | Val Loss: 0.3758 | Counter: 0
Epoch 3/30 | Train Loss: 0.3403 | Val Loss: 0.3290 | Counter: 0
Epoch 4/30 | Train Loss: 0.3110 | Val Loss: 0.3302 | Counter: 1
Epoch 5/30 | Train Loss: 0.2885 | Val Loss: 0.3119 | Counter: 0
Epoch 6/30 | Train Loss: 0.2724 | Val Loss: 0.2984 | Counter: 0
Epoch 7/30 | Train Loss: 0.2574 | Val Loss: 0.3085 | Counter: 1
Epoch 8/30 | Train Loss: 0.2417 | Val Loss: 0.3114 | Counter: 2
Epoch 9/30 | Train Loss: 0.2314 | Val Loss: 0.3038 | Counter: 3
Epoch 10/30 | Train Loss: 0.2179 | Val Loss: 0.3142 | Counter: 4
Epoch 11/30 | Train Loss: 0.2055 | Val Loss: 0.3064 | Counter: 5
Epoch 12/30 | Train Loss: 0.1660 | Val Loss: 0.2896 | Counter: 0
Epoch 13/30 | Train Loss: 0.1569 | Val Loss: 0.3037 | Counter: 1
Epoch 14/30 | Train Loss: 0.1469 | Val Loss: 0.3041 | Counter: 2
Epoch 15/30 | Train Loss: 0.1392 | Val Loss: 0.3064 | Counter: 3
Epoch 16/30 | Train Loss: 0.1287 |

In [ ]:
Fashion_ANN.eval() 

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device) 
        outputs = Fashion_ANN(images) 
        _, predicted = torch.max(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 90.17%
